<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_7_part_2/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [7]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [8]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [9]:
from surprise import Dataset, SVD, Reader

In [10]:
!pip install -q surprise

In [11]:
reader = Reader(rating_scale=(0.5, 5))

In [12]:
data = Dataset.load_from_df(df[["blog_id", "userId", "ratings"]], reader)

In [13]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [14]:

from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [15]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [16]:
from surprise import KNNBasic

algo = KNNBasic(k=10,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [17]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [18]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

Estimating biases using als...
mae = 1.6490275822780145
mse = 3.477186488384697
rmse= 1.8647215578698866
fcp = 0.4987313847422496


In [19]:
algo.predict(uid=2,
             iid=97)

Prediction(uid=2, iid=97, r_ui=None, est=4.508657858770636, details={'was_impossible': False})

In [21]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [22]:
import pandas as pd
from surprise import Dataset, SVD, Reader, accuracy
from surprise.model_selection import train_test_split, cross_validate
import optuna

# Загрузка данных
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

# Вывод первых строк для проверки
print(df.head())


   blog_id  userId  ratings
0     9025      11      3.5
1     9320      11      5.0
2     9246      11      3.5
3     9431      11      5.0
4      875      11      2.0


In [23]:

reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(df[["blog_id", "userId", "ratings"]], reader)

trainset, testset = train_test_split(data, test_size=0.2)


In [24]:
model = SVD()
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8368  1.8342  1.8257  1.8333  1.8329  1.8326  0.0037  
MAE (testset)     1.6348  1.6333  1.6248  1.6319  1.6324  1.6314  0.0035  
Fit time          5.14    4.89    4.59    4.69    7.65    5.39    1.14    
Test time         0.74    0.57    0.29    1.11    0.72    0.69    0.27    


{'test_rmse': array([1.83683064, 1.83422082, 1.8256785 , 1.8332637 , 1.83292162]),
 'test_mae': array([1.63483404, 1.63329251, 1.62479927, 1.6318718 , 1.63236957]),
 'fit_time': (5.140553712844849,
  4.891180992126465,
  4.586735248565674,
  4.694289922714233,
  7.6485254764556885),
 'test_time': (0.7419443130493164,
  0.5681638717651367,
  0.2888069152832031,
  1.1083214282989502,
  0.7231965065002441)}

In [25]:
def objective(trial):
    n_factors = trial.suggest_int('n_factors', 10, 100)
    n_epochs = trial.suggest_int('n_epochs', 10, 50)
    lr_all = trial.suggest_float('lr_all', 0.001, 0.1)
    reg_all = trial.suggest_float('reg_all', 0.02, 0.2)

    model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

    results = cross_validate(model, data, measures=['RMSE'], cv=3, verbose=False)

    return results['test_rmse'].mean()

In [26]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

study.best_params

[I 2024-09-09 19:29:59,575] A new study created in memory with name: no-name-90c62e1b-2ed8-430d-9eb5-74a7f1d96856
[I 2024-09-09 19:30:13,606] Trial 0 finished with value: 1.8875587094776154 and parameters: {'n_factors': 97, 'n_epochs': 17, 'lr_all': 0.051781271608670375, 'reg_all': 0.04711010000735133}. Best is trial 0 with value: 1.8875587094776154.
[I 2024-09-09 19:30:30,381] Trial 1 finished with value: 1.9024319995070638 and parameters: {'n_factors': 51, 'n_epochs': 26, 'lr_all': 0.07890496966326904, 'reg_all': 0.1243011109974746}. Best is trial 0 with value: 1.8875587094776154.
[I 2024-09-09 19:30:51,782] Trial 2 finished with value: 1.8863738111802804 and parameters: {'n_factors': 49, 'n_epochs': 35, 'lr_all': 0.08178575384011885, 'reg_all': 0.1829785165016337}. Best is trial 2 with value: 1.8863738111802804.
[I 2024-09-09 19:31:04,117] Trial 3 finished with value: 1.9651307042146993 and parameters: {'n_factors': 31, 'n_epochs': 22, 'lr_all': 0.014132499284689258, 'reg_all': 0.05

{'n_factors': 10,
 'n_epochs': 10,
 'lr_all': 0.0015445240743580074,
 'reg_all': 0.13836845697925265}

In [27]:
best_model = SVD(**study.best_params)
best_model.fit(trainset)
predictions = best_model.test(testset)
print("Лучший RMSE на тестовом наборе данных:", accuracy.rmse(predictions))

RMSE: 1.7728
Лучший RMSE на тестовом наборе данных: 1.772806567089823
